In [1]:
%use dataframe(0.9.1)
// @file:DependsOn("E:\\Presentations\\dependencies\\dataframe-0.9.1.jar")
// @file:DependsOn("E:\\Presentations\\dependencies\\commons-csv-1.10.0.jar")
import java.time.LocalDate
import java.time.format.DateTimeFormatter
import kotlinx.datetime.LocalDate as KDate
import org.apache.commons.csv.CSVFormat

In [2]:
val csvpath : String = "E:\\Presentations\\KotlinConfDataframe\\assets2\\data\\"

val dfCalendars = DataFrame.readCSV("${csvpath}calendars.csv", delimiter = ';')
val dfHotelOffers = DataFrame.readCSV("${csvpath}hoteloffers.csv", delimiter = ';')
val dfTierOffers = DataFrame.readCSV("${csvpath}tierofferssimple.csv", delimiter = ';')
val dfPlayers = DataFrame.readCSV("${csvpath}players.csv", delimiter = ';')
val dfMessages = DataFrame.readCSV("${csvpath}msg.csv", delimiter = ';')
val dfTiers = DataFrame.readCSV("${csvpath}tierdefinitionssimple.csv", delimiter = ';')

In [3]:
dfTiers

DataFrame: rowsCount = 16, columnsCount = 6

In [4]:
dfMessages

DataFrame: rowsCount = 8, columnsCount = 6

In [7]:
// dfMessages provides a TierMin & TierMax
// with explode(), we can create a row for every individual tier in the range
// then we can innerJoin with dfTiers, now every Message is matched with each of the corresponding tiers
val dfMsgTiers = dfMessages
    .remove { Category }
    .add("Tier") { (TierMin..TierMax).toList() }
    .explode("Tier")
    .innerJoin(dfTiers) { "Tier"<Int>() match(right.Tier) }
    .remove { "OfferMonth1" and TierMin and TierMax }
    .move("Tier").after { OfferMonth }

In [8]:
dfMsgTiers
    .filter { Tier == 2003 }
    .sortBy { Placement and Tier }
    
    
    //.sortBy { Placement and Tier }
    
    
    // .sortBy { Placement and Tier }

DataFrame: rowsCount = 5, columnsCount = 8

In [9]:
val dfMsgPivot = dfMsgTiers
    .groupBy { OfferMonth and Tier }
        .pivot { Placement }
            .with { Message }
    .flatten()
    
dfMsgPivot

DataFrame: rowsCount = 16, columnsCount = 7

In [10]:
dfTierOffers

DataFrame: rowsCount = 16, columnsCount = 10

In [11]:
val GamePref by column<String>()
val GamingAmount by column<Int>()

val dfTierOffersByGame = dfTierOffers
    .add(GamePref) { listOf("Slot", "Table") }
    .explode { GamePref }
    .add(GamingAmount) { if (GamePref() == "Slot") SlotAmount else TableAmount }
    .update { TableType }.with { if (GamePref() == "Slot") "Slot Credits" else it }
    .rename { TableType }.into("GamingType")
    .move { GamePref and GamingAmount }.after { Tier }
    .remove { SlotAmount and TableAmount }
    
dfTierOffersByGame

... showing only top 20 of 32 rows DataFrame: rowsCount = 32, columnsCount = 10

In [12]:
val dfTieredOffersPlusHotel = dfTierOffersByGame
    .leftJoin(dfMsgPivot) { OfferMonth and Tier }
    .leftJoin(dfHotelOffers) { OfferMonth and HotelCode }
    .update("GamingHeadline", "GamingSubhead").with { 
        it.toString()
            .replace("~~", GamingAmount.let{ g -> "\$${"%,d".format(g)}" })
            .replace("@@", GamingAmount.times(GamingFrequency).let{ g ->"\$${"%,d".format(g)}" })
            .replace("**", GamingType)
            .replace("##", LocalDate.parse(OfferMonth.toString()).format(DateTimeFormatter.ofPattern("MMMM")))
    }.update("DiningHeadline", "DiningSubhead").with { 
        it.toString()
            .replace("~~", DiningAmount.let {d -> "\$${"%,d".format(d)}" })
            .replace("@@", DiningAmount.times(DiningFrequency).let{ d ->"\$${"%,d".format(d)}" })
            .replace("##", LocalDate.parse(OfferMonth.toString()).format(DateTimeFormatter.ofPattern("MMMM")))
    }.update("OutsideHeadline").with { 
        it.toString()
            .replace("##", LocalDate.parse(OfferMonth.toString()).format(DateTimeFormatter.ofPattern("MMMM")).uppercase())
            .replace("^^", (GamingAmount.times(GamingFrequency)).plus(DiningAmount.times(DiningFrequency)).let{ t ->"\$${"%,d".format(t)}" })
    }
    .move("GamePref", "OutsideHeadline", "HotelHeadline", "HotelDescription", "HotelCode", 
             "GamingHeadline", "GamingSubhead", "DiningHeadline", "DiningSubhead").after { Tier }

    
dfTieredOffersPlusHotel

... showing only top 20 of 32 rows DataFrame: rowsCount = 32, columnsCount = 17

In [13]:
dfTierOffersByGame.describe()

DataFrame: rowsCount = 10, columnsCount = 12

In [14]:
dfCalendars

DataFrame: rowsCount = 18, columnsCount = 5

In [15]:
fun LocalDate.showAs(pattern: String) : String = this.format(DateTimeFormatter.ofPattern(pattern))
fun LocalDate.showAsOrAbbr(pattern: String, maximumMonthLetters: Int) : String =
    (if (this.showAs("MMMM").length > maximumMonthLetters) pattern.replace("MMMM", "MMM.") else pattern).let { this.showAs(it) }
fun areSameMonth(firstDate: LocalDate, secondDate: LocalDate) : Boolean = (firstDate.month == secondDate.month && firstDate.year == secondDate.year)

fun createReadableDateRange(startDate: LocalDate, endDate: LocalDate, maximumMonthLetters: Int = 12) : String =
    when {
        areSameMonth(startDate, endDate) -> listOf<String>(
                                                startDate.showAsOrAbbr("MMMM d", maximumMonthLetters),
                                                endDate.showAs("d, yyyy")
                                            ).joinToString("-")
        else -> "${startDate.showAs("MMM. d")} to ${endDate.showAs("MMM. d, yyyy")}"
    }


data class OfferCalendar(
    val OfferMonth: LocalDate,
    val CalendarName: String,
    val Period: Int,
    val StartDate: LocalDate,
    val EndDate: LocalDate
)

fun OfferCalendar.readableRange(maxMonthLetters: Int) = createReadableDateRange(StartDate, EndDate, maxMonthLetters)
fun OfferCalendar.types() = when {
    CalendarName.contains("All") -> listOf("gaming", "dining", "hotel")
    CalendarName.contains("Hotel") -> listOf("hotel")
    CalendarName.contains("Gaming") -> listOf("gaming")
    CalendarName.contains("Dining") -> listOf("dining")
    else -> listOf()
}

In [16]:
val calendarList : List<OfferCalendar> = dfCalendars.toListOf<OfferCalendar>()
calendarList

[OfferCalendar(OfferMonth=2023-06-01, CalendarName=HotelCalendar, Period=1, StartDate=2023-06-01, EndDate=2023-06-15), OfferCalendar(OfferMonth=2023-06-01, CalendarName=HotelCalendar, Period=2, StartDate=2023-06-16, EndDate=2023-06-30), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=1, StartDate=2023-06-01, EndDate=2023-06-03), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=2, StartDate=2023-06-04, EndDate=2023-06-06), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=3, StartDate=2023-06-07, EndDate=2023-06-09), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=4, StartDate=2023-06-10, EndDate=2023-06-12), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=5, StartDate=2023-06-13, EndDate=2023-06-15), OfferCalendar(OfferMonth=2023-06-01, CalendarName=GamingCalendar, Period=6, StartDate=2023-06-16, EndDate=2023-06-18), OfferCalendar(OfferMonth=2023-06-01, CalendarName

In [17]:
val ValidDates by column<List<OfferCalendar>>()
val ValidDateStrings by column<List<String>>()

val dfTieredOffersAndDates = dfTieredOffersPlusHotel
    // we got to here in an earlier cell
    // now we're going to bring in OfferCalendar objects, and assign a list of them (based on geo) to each tier
    .add(ValidDates) { dfCalendars.filter { when {
                                                    Tier <= 1999 -> !CalendarName.contains("Geo")
                                                    Tier >= 2000 -> CalendarName.startsWith("CalGeo")
                                                    else -> true
                                                } }.toListOf<OfferCalendar>() }
    // filter each specific type of OfferCalendar objects into their own columns (each contains a list)
    .add {
        "HotelOffers" from ValidDates.map { it.filter { it.types().contains("hotel") } }
        "GamingOffers" from ValidDates.map { it.filter { it.types().contains("gaming") } }
        "DiningOffers" from ValidDates.map { it.filter { it.types().contains("dining") } }
    }
    // split the lists so each element is in its own column
    .split { "HotelOffers"<List<OfferCalendar>>() }.into { it -> "hotel$it" }
    .split { "GamingOffers"<List<OfferCalendar>>() }.into { it -> "gaming$it" }
    .split { "DiningOffers"<List<OfferCalendar>>() }.into { it -> "dining$it" }
    .remove { ValidDates }
    // now convert the OfferCalendar objects to the string representation of date range
    // don't forget the ? - some columns contain null values because lists were not all same size
    .convert { colsOf<OfferCalendar?>() }.with { it?.readableRange(5) ?: "" }
    
    
dfTieredOffersAndDates

... showing only top 20 of 32 rows DataFrame: rowsCount = 32, columnsCount = 33

In [18]:
val dfPlayerOffers = dfPlayers
    .sortBy { Tier and GamePref and PlayerID } 
    .update { Email }.with { it?.lowercase()?.trim() ?: "" }
    .remove { EvaluationDate and ExclusionCode and Mailing and Geo and ADT and Measure }
    .innerJoin(dfTieredOffersAndDates) { Tier and GamePref }
    .move { Tier and GamePref  }.after { OfferMonth }
    .convert { Birthday }.with { Birthday.month == OfferMonth.month }
    .remove("OfferMonth1")
    
dfPlayerOffers

... showing only top 20 of 914 rows DataFrame: rowsCount = 914, columnsCount = 43

In [19]:
dfPlayerOffers
    .remove { Email and Birthday and 
                GamingAmount and GamingFrequency and DiningAmount and DiningFrequency and HotelFrequency }
    .writeCSV("MailFile.csv", CSVFormat.DEFAULT.withDelimiter(';'))
    
dfPlayerOffers
    .filter { Email.contains("@") }
    .remove { Address and City and State and Country and ZipCode and 
                GamingAmount and GamingFrequency and DiningAmount and DiningFrequency and HotelFrequency }
    .writeCSV("EmailFile.csv", CSVFormat.DEFAULT.withDelimiter(';'))    

In [20]:
dfPlayerOffers.size()

914 x 43